In [ ]:
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf, params

import numpy as np
import os

from caiman_preprocessing import resave_data_path, get_params_dict
import src.caiman_macros as cm_macros

In [ ]:
def preprocess_movie(params_dict: dict) -> None:
    """
    Manually preprocess the data and display intermediate movies and images.
    See https://github.com/flatironinstitute/CaImAn/blob/master/demos/notebooks/demo_pipeline.ipynb
    for more details.
    :param params_dict: A dictionary of parameters.
    """

    # Play the movie of the original data
    movie_orig = cm.load(params_dict['fnames'])
    movie_orig.play()

    # Start a cluster
    dview, n_processes = cm_macros.start_cluster()

    # Save the parameter dictionary into a params.CNMFParams object
    opts = params.CNMFParams(params_dict=params_dict)

    # Perform rigid motion correction
    mc = MotionCorrect(params_dict['fnames'], dview=dview, **opts.get_group('motion'))
    mc.motion_correct_rigid(save_movie=True)

    # Compare the motion-corrected data with the original data
    movie_mc = cm.load(mc.mmap_file)
    cm.concatenate([movie_orig - mc.min_mov * mc.nonneg_movie, movie_mc], axis=2).play()

    # Save and load memory mapped files from motion correction
    base_name = params_dict['fnames'][0][:-4] + '_memmap_'
    fname_mmap = cm.save_memmap(mc.mmap_file, base_name=base_name, order='C', dview=dview)
    Yr, dims, T = cm.load_memmap(fname_mmap)
    images = np.reshape(Yr.T, [T] + list(dims), order='F')

    # Restart the cluster to clean up memory
    dview, n_processes = cm_macros.start_cluster(dview)

    # Run CNMF
    cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
    cnm = cnm.fit(images)

    # Plot the contours of any identified components against the correlation image
    corr_img = cm.local_correlations(images.transpose((1, 2, 0)))
    corr_img[np.isnan(corr_img)] = 0
    cnm.estimates.plot_contours_nb(img=corr_img)

    # Rerun CNMF
    cnm = cnm.refit(images, dview=dview)

    # Evaluate the quality of inferred spatial components
    cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

    # Display plots of accepted and rejected components
    cnm.estimates.plot_contours_nb(img=corr_img, idx=cnm.estimates.idx_components)
    cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components)
    if len(cnm.estimates.idx_components_bad) > 0:
        cnm.estimates.nb_view_components(img=corr_img, idx=cnm.estimates.idx_components_bad)

    # Keep only the accepted components
    cnm.estimates.select_components(use_object=True)

    # View the final results
    cnm.estimates.nb_view_components(img=corr_img, denoised_color='red')
    cnm.estimates.play_movie(images)

    # Save results
    cnm.save(cnm.mmap_file[:-4] + 'hdf5')

    # Stop the cluster
    cm.stop_server(dview=dview)

In [ ]:
# Load and resave the data
# See GitHub Issue #377 - https://github.com/flatironinstitute/CaImAn/issues/377#issuecomment-426740429
# resave_data_path()

os.chdir('C:/Users/pugavin/BCB-330-2023-05/pugavin/results')
fnames = ['F147.tif', 'F201.tif']

# Retrieve parameters
params_dict = get_params_dict()

# Analyze all files
for fname in fnames:

    # Add the file to the parameter dictionary
    params_dict['fnames'] = [fname]

    # Begin preprocessing
    preprocess_movie(params_dict)